## Лекция 3  NER

### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)

In [85]:
!pip install yargy

In [86]:
from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

In [87]:
!pip install natasha

In [88]:
from natasha import NamesExtractor
import re

In [111]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    
    Doc
)

In [90]:
!pip install git+https://github.com/deepmipt/ner
import ner
extractor = ner.Extractor()

  Cloning https://github.com/deepmipt/ner to /tmp/pip-req-build-ftqlfsri
  Running command git clone -q https://github.com/deepmipt/ner /tmp/pip-req-build-ftqlfsri
  Created wheel for ner: filename=ner-0.0.1-cp36-none-any.whl size=22531 sha256=1593526e7e43a823a3078277b0dc1f230686c5c8a69deabf309b8272e6e4831a
  Stored in directory: /tmp/pip-ephem-wheel-cache-jjjrzid5/wheels/46/f5/1c/0657f016f0e9725ee09f56dab547bd0bcb76fbbbc067a950ea
Successfully built ner


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /usr/local/lib/python3.6/dist-packages/ner/extractor/../model/ner_model


In [91]:
def preprocess_with_natasha(text: str) -> str:
  segmenter = Segmenter()
  emb = NewsEmbedding()
  morph_tagger = NewsMorphTagger(emb)
  syntax_parser = NewsSyntaxParser(emb)

  doc = Doc(text)

  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)

  list_of_needed = {}

  sent = doc.sents[0]
  for ss in sent.morph:
    for s in ss:
      search1 = re.findall('text=\'(.*?)\'',str(s))
      search2 = re.findall('pos=\'(.*?)\'',str(s))
      list_of_needed[' '.join(search1)] = ' '.join(search2)

  for k,v in list_of_needed.items():
    if v == 'PROPN':
      text = re.sub(k, '', text)
  return text

def preprocess_with_deepmipt(text: str) -> str:
  list_of_extractors = []
  for m in extractor(text):
    for mm in m:
      if str(type(mm)) == '<class \'list\'>':
        list_of_extractors.append(mm)
  list_2 = []
  for l in list_of_extractors:
    word = re.findall('text=\'(.*?)\'', str(l))
    if type(word) is list:
      list_2 = list_2 + word
    else:
      list_2.append(' '.join(word))
  for word in text.split():
    for ww in list_2:
      if word == ww:
        text = re.sub(word, '', text)
  return text

In [92]:
preprocess_with_natasha('До Москвы недалеко')

'До  недалеко'

In [93]:
preprocess_with_deepmipt('До Москвы недалеко')

'До  недалеко'

### __Задача 2__:    
На предыдущем занятии вы реализовывали функции поиска ближайших ответов на запросы через TF-IDF и BM25. 
Сравните качество нахождения верного ответа для обоих методов в трех случаях:
- с функцией ```preprocess_with_natasha```
- с функцией ```preprocess_with_deepmipt```
- без препроцессинга

Для измерения качества используйте метрику accuracy. Считаем, что ответ верный, если он входит в топ-1.

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# инициализируем
vectorizer = TfidfVectorizer()

In [127]:
import re
import pandas as pd
import unicodedata

answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel('queries_base.xlsx')

In [128]:
queries

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
...,...,...,...,...,...
2294,Добрый день.\nГражданин Украины проживает в РФ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2295,Добрый вечер. Нахожусь в командировке за грани...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2296,Здравствуйте! \nНам пришёл положительный резул...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
2297,"Добрый день, хотелось бы получить консультацию...",45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ\n",NaN,NaN


In [129]:
new_seventy = queries[:int(len(queries)*0.7)]

In [130]:
other_seventy = queries[1609:]#:-int(len(queries)*0.7)]
other_seventy

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4
1609,Меня интересует конкретно переход Границы с Аб...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1610,Добрый день.\n26.08 прибыл из Турции на террит...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1611,"Добрый день!Интересует такой вопрос, ко мне в ...",308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1612,Добрый день!\nУ члена семьи был подтвержден ко...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
1613,Добрый день. Я не нашла ответа на свой вопрос ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
...,...,...,...,...,...
2294,Добрый день.\nГражданин Украины проживает в РФ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2295,Добрый вечер. Нахожусь в командировке за грани...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2296,Здравствуйте! \nНам пришёл положительный резул...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
2297,"Добрый день, хотелось бы получить консультацию...",45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ\n",NaN,NaN


In [131]:
new_seventy

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
...,...,...,...,...,...
1604,Здравствуйте меня зовут Александр Архипов. Я г...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1605,Результат Пцр мазка Мещанинова Константина Вал...,6.0,ТЕСТИРОВАНИЕ (АНАЛИЗ) НА КОРОНАВИРУС\n,NaN,NaN
1606,Я являюсь сотрудником министерства гражданской...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
1607,"Здравствуйте!Мне нужно уточнить по вопросу, ка...",308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN


In [132]:
sevvo_4 = pd.DataFrame()
sevvo_4['Текст вопроса'] = pd.concat([new_seventy['Текст вопроса'],answers['Текст вопросов']],ignore_index=True)
sevvo_4['Номер связки'] = pd.concat([new_seventy['Номер связки\n'],answers['Номер связки']],ignore_index=True)

sevvo_4

,Текст вопроса,Номер связки
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0
3,Добрый день.\nВ Кемеровской области согласно п...,45.0
4,"Здравствуйте, в моем городе Кострома введено о...",3.0
...,...,...
1647,Платные тесты на ковид?\nГде сдать тест на ков...,135.0
1648,"Мне 65 лет, куда обращаться, если работодатель...",5.0
1649,Куда жаловаться (обращаться) если я вижу наруш...,3.0
1650,Рекомендации для кафе и ресторанов?\nРекоменда...,45.0


In [133]:
n_conn = sevvo_4['Номер связки'].tolist()
other_seventy

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4
1609,Меня интересует конкретно переход Границы с Аб...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1610,Добрый день.\n26.08 прибыл из Турции на террит...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1611,"Добрый день!Интересует такой вопрос, ко мне в ...",308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1612,Добрый день!\nУ члена семьи был подтвержден ко...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
1613,Добрый день. Я не нашла ответа на свой вопрос ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
...,...,...,...,...,...
2294,Добрый день.\nГражданин Украины проживает в РФ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2295,Добрый вечер. Нахожусь в командировке за грани...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2296,Здравствуйте! \nНам пришёл положительный резул...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
2297,"Добрый день, хотелось бы получить консультацию...",45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ\n",NaN,NaN


In [134]:
X = vectorizer.fit_transform(sevvo_4['Текст вопроса'].values.astype('U'))
Y = vectorizer.transform(other_seventy['Текст вопроса'].values.astype('U')).toarray()

Y_t = Y.transpose()
X.dot(Y_t)

array([[0.        , 0.04287689, 0.05834729, ..., 0.03167038, 0.01363547,
        0.00321257],
       [0.        , 0.09973354, 0.01685706, ..., 0.07604975, 0.00977253,
        0.        ],
       [0.00959547, 0.10001181, 0.06739524, ..., 0.03198831, 0.02917185,
        0.0350019 ],
       ...,
       [0.01104936, 0.04753727, 0.05754338, ..., 0.00094292, 0.02671735,
        0.01690017],
       [0.        , 0.02815584, 0.03112915, ..., 0.00125265, 0.01136519,
        0.00424314],
       [0.        , 0.01002816, 0.06019289, ..., 0.        , 0.00801717,
        0.        ]])

In [135]:
import operator

def ranging(text):
  dict_texts = {}
  cnt = 0
  for x in np.nditer(text):
    dict_texts[n_conn[cnt]] = float(x)
    cnt += 1
  new_dict = sorted(dict_texts.items(),
                    key = operator.itemgetter(1), reverse = True)
  first = []
  for k in new_dict:
    first.append(k[0])
  return first[0]

ranging_list = []

for o in X.dot(Y_t):
  ranging_list.append(ranging(o))

other_seventy['tf_none'] = ranging_list[:690]

accuracy = 0
for s, t in zip(other_seventy['Номер связки\n'],other_seventy['tf_none']):
  if s == t:
    accuracy += 1

acc = accuracy/len(other_seventy['Номер связки\n'])
print(acc)

0.04492753623188406


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [119]:
train_frame = pd.DataFrame()
old_frame = pd.DataFrame()

hyper_list = []
test_list = []

for t in sevvo_4['Текст вопроса']:
  hyper_list.append(preprocess_with_natasha(str(t)))

for o in other_seventy['Текст вопроса']:
  test_list.append(preprocess_with_natasha(str(o)))

train_frame['Текст вопросов'] = hyper_list
train_frame['Номер связки'] = sevvo_4['Номер связки']

old_frame['Текст вопросов'] == test_list
old_frame['Номер связки'] = other_seventy['Номер связки\n']

n_conn = train_frame['Номер связки'].tolist()

X = vectorizer.fit_transform(train_frame['Текст вопроса'].values.astype('U'))
Y = vectorizer.transform(other_seventy['Текст вопроса'].values.astype('U')).toarray()

Y_t = Y.transpose()
#X.dot(Y_t)

ranging_list = []

for o in X.dot(Y_t):
  ranging_list.append(ranging(o))

other_seventy['tf_natasha'] = ranging_list[:690]

accuracy = 0
for s, t in zip(other_seventy['Номер связки\n'],other_seventy['tf_natasha']):
  if s == t:
    accuracy += 1

acc = accuracy/len(other_seventy['Номер связки\n'])
print(acc)


In [ ]:
train_frame = pd.DataFrame()
old_frame = pd.DataFrame()

hyper_list = []
test_list = []

for t in sevvo_4['Текст вопроса']:
  hyper_list.append(preprocess_with_deepmipt(str(t)))

for o in other_seventy['Текст вопроса']:
  test_list.append(preprocess_with_deepmipt(str(o)))

train_frame['Текст вопросов'] = hyper_list
train_frame['Номер связки'] = sevvo_4['Номер связки']

old_frame['Текст вопросов'] == test_list
old_frame['Номер связки'] = other_seventy['Номер связки\n']

n_conn = train_frame['Номер связки'].tolist()

X = vectorizer.fit_transform(train_frame['Текст вопроса'].values.astype('U'))
Y = vectorizer.transform(other_seventy['Текст вопроса'].values.astype('U')).toarray()

Y_t = Y.transpose()
#X.dot(Y_t)

ranging_list = []

for o in X.dot(Y_t):
  ranging_list.append(ranging(o))

other_seventy['tf_deepmint'] = ranging_list[:690]

accuracy = 0
for s, t in zip(other_seventy['Номер связки\n'],other_seventy['tf_deepmint']):
  if s == t:
    accuracy += 1

acc = accuracy/len(other_seventy['Номер связки\n'])
print(acc)


In [165]:
### реализуйте эту функцию ранжирования 
from math import log
import statistics
import re

k = 2.0
b = 0.75

collection = list(sevvo_4['Текст вопроса'])
#collection_2 = list(other_seventy['Текст вопроса'])

collection_length = []
n_big = len(collection)

for c in collection:
  collection_length.append(len(c))

avgdl = statistics.mean(collection_length)
#avgdl_2 = statistics.mean(collection_2_length)

# d - один документ
# q - набор слов

def bm25(d,q) -> float:
  l_d = len(d)
  def frequency(single_q):
    tf_q_d = 0
    for word in d.split():
      word = re.sub('!@#$%^&*\(\)\[\]\{\}.,><?/-/"',' ',word)
      word = word.lower()
      #print(word)
      #print(single_q)
      if single_q == word:
        tf_q_d += 1
    return tf_q_d
  
  def get_idf(single_q):
    
    def number_of_docs(another_q):
      doc_num = 0
      for col in collection:
        if another_q in col:
          doc_num += 1
      return doc_num
  
    end_result = log((n_big - number_of_docs(single_q) + 0.5)/
                     (number_of_docs(single_q) + 0.5))
    return end_result
  
  final_meanings = []

  for query in q:
    #print(query)
    upper_half = frequency(query) * (k + 1)
    lower_half = frequency(query) + k * (1 - b + b * (l_d)/avgdl)
    final_meanings.append(get_idf(query) * upper_half/lower_half)

  return sum(final_meanings)

for bm25()

TypeError: ignored

### __Задача 3__:    
Улучшить правила в natasha. Написать правила, которые ловят даты в следующих примерах и пересчитать статистику из Задачи 2:
- Уехал 8-9 ноября в Сочи
- Уезжаю 5 числа                           
- 20го сентября заболел

Пример можно посмотреть тут: https://github.com/natasha/yargy

In [117]:
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte

def preprocess_with_natasha_updated(text: str) -> str:
  segmenter = Segmenter()
  emb = NewsEmbedding()
  morph_tagger = NewsMorphTagger(emb)
  syntax_parser = NewsSyntaxParser(emb)
  
  doc = Doc(text)

  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)

  MIDNUM_1 = and_(
    gte(0),
    lte(9)
  )
  MIDNUM_2 = and_(
    gte(0),
    lte(9)
  )
  MIDNUM = rule(
    MIDNUM_1,
    '-',
    MIDNUM_2,
  )
  SEPNUM = rule(
      MIDNUM_1,
      ' '
  )
  DATE_CASUAL = rule(
      MIDNUM_1,
      '-',
      {'го',
       'ой',
       'ей',
       'м',
       'ым',
       'ых',
       'х'}
  )
  DATE_CLOSER = rule(
      MIDNUM_1,
      {'го',
       'ой',
       'ей',
       'м',
       'ым',
       'ых',
       'х'}
  )

  list_of_needed = {}

  sent = doc.sents[0]

  for ss in sent.morph:
    for s in ss:
      search1 = re.findall('text=\'(.*?)\'',str(s))
      search2 = re.findall('pos=\'(.*?)\'',str(s))
      list_of_needed[' '.join(search1)] = ' '.join(search2)
  
  for k,v in list_of_needed.items():
    if v == 'PROPN':
      text = re.sub(k, '', text)
  return text


  

In [118]:
train_frame = pd.DataFrame()
old_frame = pd.DataFrame()

hyper_list = []
test_list = []

for t in sevvo_4['Текст вопроса']:
  hyper_list.append(preprocess_with_natasha_updated(str(t)))

for o in other_seventy['Текст вопроса']:
  test_list.append(preprocess_with_natasha_updated(str(o)))

train_frame['Текст вопросов'] = hyper_list
train_frame['Номер связки'] = sevvo_4['Номер связки']

old_frame['Текст вопросов'] == test_list
old_frame['Номер связки'] = other_seventy['Номер связки\n']

n_conn = train_frame['Номер связки'].tolist()

X = vectorizer.fit_transform(train_frame['Текст вопроса'].values.astype('U'))
Y = vectorizer.transform(other_seventy['Текст вопроса'].values.astype('U')).toarray()

Y_t = Y.transpose()
#X.dot(Y_t)

ranging_list = []

for o in X.dot(Y_t):
  ranging_list.append(ranging(o))

other_seventy['tf_natasha_updated'] = ranging_list[:690]

accuracy = 0
for s, t in zip(other_seventy['Номер связки\n'],other_seventy['tf_natasha_updated']):
  if s == t:
    accuracy += 1

acc = accuracy/len(other_seventy['Номер связки\n'])
print(acc)


DocSent(stop=23, text='Уехал 8-9 ноября в Сочи', tokens=[...])


'Уехал 8-9 ноября в '